In [7]:
import pandas as pd
import json

* Read the results of the complete ALNS executions

In [8]:
with open("alns_results_real.json", "r") as file:
    alns_results = json.load(file)

* Compute averages per trial, max and min trial for each instance

In [ ]:
alns_stats= {}
for instance, instance_data in alns_results.items():
    results = instance_data.get("method:", {}).get("ALNS", {})

    # compute best makespan across all instances in the same group (same S_x and _Ord_y)
    group_prefix = instance.split('-I_')[0]
    all_makespan_same_group = [
        trial_data["best_cost"]
        for inst, inst_data in alns_results.items()
        if inst.startswith(group_prefix)
        for trial_data in inst_data["method:"]["ALNS"].values()
    ]
    bests_makespan = min(all_makespan_same_group)

    makespan_values = []
    trial_metrics = []
    for trial_name, trial_data in results.items():
        makespan_values.append(trial_data["best_cost"])
        trial_metrics.append((trial_name, trial_data))
    # compute average metrics
    avg_metrics = {}
    if trial_metrics:
        keys = trial_metrics[0][1].keys()
        for key in keys:
            if key.endswith("parameters") or key.endswith("best_solution") or key.endswith("final_operator_weights"):
                continue
            avg_metrics[key] = sum(m[key] for _, m in trial_metrics) / len(trial_metrics)
        # compute average RPD
        rpd_values = [(m["best_cost"] - bests_makespan) / bests_makespan for _, m in trial_metrics]
        avg_metrics["RPD"] = sum(rpd_values) / len(rpd_values) if rpd_values else None
    # find max and min fitness trials
    max_idx = makespan_values.index(max(makespan_values))
    min_idx = makespan_values.index(min(makespan_values))
    max_trial = trial_metrics[max_idx][1]
    min_trial = trial_metrics[min_idx][1]
    alns_stats[instance] = {
        "average": avg_metrics,
        "max_makespan_trial": max_trial,
        "min_makespan_trial": min_trial
}

* Save the detailed results

In [10]:
if alns_stats != {}:
    with open("alns_analysis.json", 'w') as file:
        json.dump(alns_stats, file, indent=4)

* Create a dataframe with the averages 

In [11]:
data = {}
for instance, instance_metrics in alns_stats.items():
    row = {}

    for metric, value in instance_metrics["average"].items():
        row[f"{metric}_average"] = value
    data[instance] = row

df_alns_avg = pd.DataFrame.from_dict(data, orient="index")

* Group the results per stages-orders categories

In [12]:
grouped = df_alns_avg.groupby(
    df_alns_avg.index.to_series().str.replace(r'-I_\d+', '', regex=True)
)

* Print the averages per stages-orders categories

In [13]:
grouped.mean().T

,S_3-Ord_10-WIP_0,S_3-Ord_30-WIP_0,S_3-Ord_50-WIP_0,S_6-Ord_10-WIP_0,S_6-Ord_30-WIP_0,S_6-Ord_50-WIP_0,S_9-Ord_10-WIP_0,S_9-Ord_30-WIP_0,S_9-Ord_50-WIP_0
best_cost_average,87.888889,477.000000,287.555556,134.111111,293.222222,604.111111,181.555556,408.222222,528.666667
iteration_found_average,85.333333,7.111111,32.111111,78.000000,92.222222,60.222222,44.444444,46.111111,54.222222
total_iterations_average,314.444444,238.333333,106.222222,360.111111,147.777778,101.111111,314.444444,154.444444,94.444444
time_found_average,27.713492,17.344918,245.431175,32.461713,331.723797,872.056096,20.883867,311.021584,1661.010476
total_time_average,103.990541,515.155668,838.363917,146.051552,576.493709,1540.045384,157.304277,897.819508,2579.825341
RPD_average,0.331650,0.084091,0.005439,0.396991,0.357510,0.585593,0.025738,0.422377,0.380331
